In [5]:
def get_model():
    from keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

    conv_kerr_size = (3, 3)
    maxpool_kerr_size = (2, 2)
    model = Sequential()
    model.add(Conv2D(32, conv_kerr_size, activation='relu', input_shape=(144, 144, 1), padding='same'))
    model.add(MaxPooling2D(maxpool_kerr_size))
    model.add(BatchNormalization())

    model.add(Conv2D(64, conv_kerr_size, activation='relu'))
    model.add(MaxPooling2D(maxpool_kerr_size))
    model.add(BatchNormalization())

    model.add(Conv2D(128, conv_kerr_size, activation='relu'))
    model.add(MaxPooling2D(maxpool_kerr_size))
    model.add(BatchNormalization())

    model.add(Conv2D(256, conv_kerr_size, activation='relu'))
    model.add(MaxPooling2D(maxpool_kerr_size))
    model.add(BatchNormalization())

    # model.add(Conv2D(256, conv_kerr_size, activation='relu'))
    # model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                      metrics=['acc'])
    return model

model = get_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 144, 144, 32)      320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 72, 72, 32)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 72, 72, 32)        128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 70, 70, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 35, 35, 64)        256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 33, 33, 128)      

In [6]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

img_size = (96, 96)
img_stat_gen = ImageDataGenerator()
train_stat_gen = img_stat_gen.flow_from_directory('../data/stretched/train', target_size=img_size,
                                                  batch_size=1, color_mode='grayscale')

all_imgs = np.zeros((28709,) + img_size + (1,)).astype('uint16')
for i, (stat_imgs, _) in enumerate(train_stat_gen):
    img = stat_imgs[0].astype('uint8')
    all_imgs[i] = img
    if i >= 28709 - 1:
        break
    if i % 100 == 0:
        print(f'Step: {i}', end='\r')

Found 28709 images belonging to 7 classes.


In [7]:
img_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
img_batch_size = 128
img_datagen.fit(all_imgs)
train_generator = img_datagen.flow_from_directory('../data/histogram_stretched/train',
                                                  target_size=img_size, batch_size=img_batch_size,
                                                  color_mode='grayscale', class_mode='categorical')
validation_generator = img_datagen.flow_from_directory('../data/histogram_stretched/val/',
                                                   target_size=img_size, batch_size=img_batch_size,
                                                   color_mode='grayscale', class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [8]:
import keras.callbacks as clb
callbacks = [
    clb.ReduceLROnPlateau(monitor='val_acc', factor=0.1, min_lr=1e-7, patience=2, verbose=1),
    clb.EarlyStopping(monitor='val_acc', patience=4, verbose=1),
    clb.ModelCheckpoint(monitor='val_acc', filepath='../models/standard_model_preprocessed.h5',
                        save_best_only=True, verbose=1)
]
nr_of_train_pictures = 28709
train_steps = nr_of_train_pictures // img_batch_size + 1
nr_of_valid_pictures = 3589
valid_steps = nr_of_valid_pictures // img_batch_size + 1
history = model.fit(train_generator, steps_per_epoch=train_steps, epochs=100,
                    validation_data=validation_generator, validation_steps=valid_steps, callbacks=callbacks,
                    verbose=2
                    )

Epoch 1/100
225/225 - 77s - loss: 1.7373 - acc: 0.3551 - val_loss: 1.8062 - val_acc: 0.2427

Epoch 00001: val_acc improved from -inf to 0.24269, saving model to ../models/standard_model_preprocessed.h5
Epoch 2/100
225/225 - 77s - loss: 1.3795 - acc: 0.4675 - val_loss: 1.3977 - val_acc: 0.4531

Epoch 00002: val_acc improved from 0.24269 to 0.45305, saving model to ../models/standard_model_preprocessed.h5
Epoch 3/100
225/225 - 78s - loss: 1.2332 - acc: 0.5278 - val_loss: 1.2453 - val_acc: 0.5319

Epoch 00003: val_acc improved from 0.45305 to 0.53190, saving model to ../models/standard_model_preprocessed.h5
Epoch 4/100
225/225 - 77s - loss: 1.1211 - acc: 0.5741 - val_loss: 1.1648 - val_acc: 0.5536

Epoch 00004: val_acc improved from 0.53190 to 0.55364, saving model to ../models/standard_model_preprocessed.h5
Epoch 5/100
225/225 - 77s - loss: 1.0328 - acc: 0.6100 - val_loss: 1.1643 - val_acc: 0.5717

Epoch 00005: val_acc improved from 0.55364 to 0.57175, saving model to ../models/standard_

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b.', label='train acc')
plt.plot(epochs, val_acc, label='validation acc')
plt.legend()

plt.figure()

plt.plot(epochs, loss, '.', label='train loss')
plt.plot(epochs, val_loss, label='validation loss')
plt.legend()

plt.show()